<a href="https://colab.research.google.com/github/zlakcswair/Python_Evaluaciones_202520/blob/main/Evaluaci%C3%B3n_3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Evaluación 2 Python aplicado a la Ingeniería 202520

Docente: Miguel Ortiz Padilla

Estudiante: Wilmer David jiménez Díaz

email: wilmer.jimenez@upb.edu.co

Fecha: 09/10/2025

In [ ]:
# Paso 1.1: Importar numpy y dale el alias np
import numpy as np
import matplotlib.pyplot as plt

def filter_data_by_date(filepath, year, month, day):
    # Paso 8.1: Envuelve todo el código de tu función dentro de un bloque try
    try:
        # Paso 1.2: Cargar datos usando genfromtxt
        data = np.genfromtxt(filepath, skip_header=True, delimiter=';')

        # Paso 3.1: Define tres variables con los valores que quieres filtrar
        # Paso 3.2: Filtrar datos por fecha usando máscaras booleanas
        filtered_data = data[(data[:, 0] == year) & (data[:, 1] == month) & (data[:, 2] == day)]

        # Paso 5.3: Validación si no hay datos
        if len(filtered_data) == 0:
            print(f"No se encontraron datos para la fecha {day}/{month}/{year}")
            return

        # Paso 9.1 y 9.2: Información adicional
        print(f"Registros cargados: {len(filtered_data)}")
        if len(filtered_data) > 0:
            temp_promedio = np.mean(filtered_data[:, 4])
            ghi_maxima = np.max(filtered_data[:, 5])
            print(f"Temperatura promedio: {temp_promedio:.2f} °C")
            print(f"Radiación máxima: {ghi_maxima:.1f} W/m2")

        # Paso 6.1: Crear nombre del archivo con formato específico
        filename = f"filtered_data_{year}_{month:02d}_{day:02d}.csv"

        # Paso 6.2: Utiliza el gestor de contexto with open()
        with open(filename, 'w') as datafile:
            # Paso 6.3.1: Escribir encabezado exacto
            datafile.write("Year,Month,Day,Hour,Temperature,GHI\n")

            # Paso 6.3.2 y 6.3.3: Escribir cada fila filtrada
            for row in filtered_data:
                datafile.write(f"{int(row[0])},{int(row[1])},{int(row[2])},{int(row[3])},{row[4]:.2f},{row[5]:.2f}\n")

        # Paso 6.4: Mensaje de confirmación
        print(f"El archivo {filename} fue creado exitosamente!")

    # Paso 8.2: Agregar tres bloques except específicos
    except FileNotFoundError:
        print(f"Error: No se encontró el archivo {filepath}")
    except IndexError:
        print("Error: El archivo no tiene el formato esperado")
    except Exception as e:
        print(f"Error inesperado: {e}")


def energy_calculations(filepath, year, month, day, n_panels, panel_area, panel_efficiency, inverter_efficiency, stc_temp=25, temp_coeff=-0.004):
    # Paso 7.2: Definir nombres de archivos adecuados
    filtered_data_filename = f"filtered_data_{year}_{month:02d}_{day:02d}.csv"
    energy_data_filename = f"energy_data_{year}_{month:02d}_{day:02d}.csv"

    # Paso 7.2: Invocar función de filtrado de datos
    filter_data_by_date(filepath, year, month, day)

    # Paso 1.2: Cargar datos filtrados generados en una variable llamada ghi_temp_data
    ghi_temp_data = np.genfromtxt(filtered_data_filename, skip_header=True, delimiter=',')

    if len(ghi_temp_data) == 0:
        print("No hay datos para procesar")
        return

    # Paso 2.1: Extraer columnas específicas (hours, temp, ghi)
    hours = ghi_temp_data[:, 3]
    temp = ghi_temp_data[:, 4]
    ghi = ghi_temp_data[:, 5]

    # Paso 3.1: Definir parámetros del sistema PV
    total_area = panel_area * n_panels

    # Paso 4.1: Calcular corrección por temperatura
    temp_correction = 1 + temp_coeff * (temp - stc_temp)

    # Paso 5.1: Calcular potencia DC
    dc_power = ghi * total_area * panel_efficiency * temp_correction

    # Paso 5.2: Calcular potencia AC
    ac_power = dc_power * inverter_efficiency

    # Paso 6.1: Calcular energía horaria
    hourly_energy_kwh = ac_power / 1000

    # Paso 6.2: Calcular energía total
    total_energy_kwh = np.sum(hourly_energy_kwh)

    # Paso 6.3: Crear archivo de datos de energía
    with open(energy_data_filename, 'w') as datafile:
        datafile.write("Hour,Power_DC,Power_AC,Energy_KWH\n")
        for i in range(len(hours)):
            datafile.write(f"{hours[i]},{dc_power[i]:.2f},{ac_power[i]:.2f},{hourly_energy_kwh[i]:.2f}\n")

    print(f"El archivo {energy_data_filename} fue creado exitosamente!")
    print(f"Registros cargados: {len(hours)}")
    print(f"Potencia DC promedio: {np.mean(dc_power):.2f} W")
    print(f"Potencia AC promedio: {np.mean(ac_power):.2f} W")
    print(f"Energía total generada: {total_energy_kwh:.2f} kWh")

    # Bonus: Gráfica de energía generada en función de la hora
    plt.figure(figsize=(10, 6))
    plt.plot(hours, hourly_energy_kwh, 'b-o', linewidth=2, markersize=6)
    plt.title(f'Energía Generada por Hora - {day}/{month}/{year}')
    plt.xlabel('Hora del día')
    plt.ylabel('Energía (kWh)')
    plt.grid(True, alpha=0.3)
    plt.xlim(0, 24)
    plt.xticks(range(0, 25, 2))
    plt.tight_layout()

    plot_filename = f"energia_vs_hora_{year}_{month:02d}_{day:02d}.png"
    plt.savefig(plot_filename, dpi=300, bbox_inches='tight')
    plt.show()
    print(f"Gráfica guardada como: {plot_filename}")


if __name__ == "__main__":
    # Paso 10.1: Crear un bloque if __name__ == "__main__": y realizar pruebas
    print("=== PRUEBAS PARTE 1 ===")

    # Paso 2.1 y 2.2: Verificar el tamaño del archivo (ejemplo con datos completos)
    print("\nPaso 2: Verificar tamaño del archivo completo:")
    data_completa = np.genfromtxt("GHI_Temp_Daily_Hourly_2024.csv", skip_header=True, delimiter=';')
    print(f"Tamaño del archivo: {data_completa.shape}")
    print(f"Número total de filas: {data_completa.shape[0]}")
    print(f"Número total de columnas: {data_completa.shape[1]}")

    print("\nPrueba 1: Fecha válida (1 de enero de 2024)")
    filter_data_by_date("GHI_Temp_Daily_Hourly_2024.csv", 2024, 1, 1)

    print("\nPrueba 2: Fecha inválida (30 de febrero)")
    filter_data_by_date("GHI_Temp_Daily_Hourly_2024.csv", 2024, 2, 30)

    print("\nPrueba 3: Archivo inexistente")
    filter_data_by_date("archivo_inexistente.csv", 2024, 1, 1)

    print("\nPrueba 4: Otra fecha válida (11 de diciembre de 2024)")
    filter_data_by_date("GHI_Temp_Daily_Hourly_2024.csv", 2024, 12, 11)

    print("\n" + "="*50)
    print("=== PRUEBAS PARTE 2 ===")

    # Paso 1.1: Comentar las demás instrucciones y usar filter_data_by_date
    # filter_data_by_date("GHI_Temp_Daily_Hourly_2024.csv", 2024, 1, 1)

    print("\nEjemplo manual de cálculos energéticos (Pasos 1-6):")

    try:
        # Paso 1.2: Cargar datos filtrados
        ghi_temp_data = np.genfromtxt("filtered_data_2024_01_01.csv", skip_header=True, delimiter=',')
        # Paso 1.3: Imprimir datos cargados y verificar tamaño
        print(f"Datos cargados: {len(ghi_temp_data)} registros")
        print(f"Tamaño del array: {ghi_temp_data.shape}")

        # Paso 2.1: Extraer columnas específicas
        hours = ghi_temp_data[:, 3]
        temp = ghi_temp_data[:, 4]
        ghi = ghi_temp_data[:, 5]

        print(f"Horas: {hours}")
        print(f"Temperaturas: {temp}")
        print(f"GHI: {ghi}")

        # Paso 3.1: Definir parámetros del sistema PV
        stc_temp = 25
        temp_coeff = -0.004
        panel_area = 1.65
        panel_efficiency = 0.2
        n_panels = 10
        total_area = panel_area * n_panels
        inverter_efficiency = 0.95

        # Paso 4.1: Calcular corrección por temperatura
        temp_correction = 1 + temp_coeff * (temp - stc_temp)

        # Paso 5.1 y 5.2: Calcular potencias DC y AC
        dc_power = ghi * total_area * panel_efficiency * temp_correction
        ac_power = dc_power * inverter_efficiency

        # Paso 5.3: Imprimir resultados
        print(f"Potencia DC: {dc_power}")
        print(f"Potencia AC: {ac_power}")

        # Paso 6.1 y 6.2: Calcular energía
        hourly_energy_kwh = ac_power / 1000
        total_energy_kwh = np.sum(hourly_energy_kwh)

        print(f"Energía total generada: {total_energy_kwh:.2f} kWh")

        # Paso 6.3: Crear archivo de energía
        with open("energy_data_2024_01_01.csv", 'w') as datafile:
            datafile.write("Hour,Power_DC,Power_AC,Energy_KWH\n")
            for i in range(len(hours)):
                datafile.write(f"{hours[i]},{dc_power[i]:.2f},{ac_power[i]:.2f},{hourly_energy_kwh[i]:.2f}\n")

        print("Archivo de energía creado exitosamente!")
        print(f"Registros cargados: {len(hours)}")
        print(f"Potencia DC promedio: {np.mean(dc_power):.2f} W")
        print(f"Potencia AC promedio: {np.mean(ac_power):.2f} W")
        print(f"Energía total producida: {total_energy_kwh:.2f} kWh")

    except FileNotFoundError:
        print("No se encontró el archivo de datos filtrados. Ejecute primero el filtrado.")

    print("\n" + "="*50)
    print("=== PRUEBA DE LA FUNCIÓN COMPLETA ===")

    # Paso 8.1: Probar la función completa con el test proporcionado
    print("\nTest de la función energy_calculations:")
    energy_calculations("GHI_Temp_Daily_Hourly_2024.csv", 2024, 12, 11, 10, 1.65, 0.2, 0.95)

    print("\n" + "-"*30)
    print("Prueba adicional con 1 de enero:")
    energy_calculations("GHI_Temp_Daily_Hourly_2024.csv", 2024, 1, 1, 15, 2.0, 0.18, 0.93)

## **Pregunta Paso 2:**

**¿Cuántas filas y columnas tiene tu archivo? ¿Qué representa cada fila?**

**Respuesta:**
- **Filas:** 8,785 registros
- **Columnas:** 6 columnas

**Estructura del archivo:**
- Columna 0: Year (Año)
- Columna 1: Month (Mes)
- Columna 2: Day (Día)
- Columna 3: Hour (Hora)
- Columna 4: Temperature (Temperatura en °C)
- Columna 5: GHI (Radiación Solar Global Horizontal en W/m²)

**¿Qué representa cada fila?**
Cada fila representa un registro horario de datos meteorológicos para una fecha y hora específica del año 2024. Contiene información sobre la temperatura ambiente y la radiación solar global horizontal en ese momento específico.

---

## **Pregunta Paso 4:**

**¿Cuántos registros encontraste para esa fecha? ¿Te parece lógico ese número?**

**Respuesta:**
- **Registros encontrados:** 24 registros para cada fecha específica

**¿Te parece lógico ese número?**
**Sí, es completamente lógico** porque:

1. **Registro horario:** Los datos se registran cada hora del día
2. **24 horas = 24 registros:** Un día completo tiene 24 horas (de 0 a 23)
3. **Cobertura completa:** Esto permite tener un registro completo de las condiciones meteorológicas durante todo el día
4. **Análisis detallado:** Es ideal para sistemas fotovoltaicos ya que se puede analizar la generación de energía hora por hora

Esta frecuencia de muestreo es estándar en estaciones meteorológicas y es perfecta para:
- Análisis de patrones diarios de radiación solar
- Cálculos de energía fotovoltaica
- Estudios de variabilidad térmica diaria
- Optimización de sistemas energéticos

---

## **Análisis Adicional de Resultados**

### **Estadísticas del Dataset Completo (2024):**
- **Total de registros:** 8,785 (aproximadamente 366 días × 24 horas)
- **Temperatura promedio anual:** 28.84°C
- **Radiación máxima registrada:** 1,015.0 W/m²

### **Ejemplo de Análisis Diario (1 de enero de 2024):**
- **Registros:** 24 horas completas
- **Temperatura promedio:** 28.04°C
- **Radiación máxima:** 627.0 W/m²
- **Patrón de radiación:** Típico de un día normal con generación solar desde las 12:00 hasta las 22:00 horas

### **Validación del Modelo Fotovoltaico:**
Los cálculos energéticos muestran resultados coherentes:
- **Energía diaria típica:** 13-21 kWh dependiendo de las condiciones
- **Potencia máxima:** Durante las horas de mayor radiación (mediodía)
- **Corrección por temperatura:** Funciona correctamente reduciendo la eficiencia a altas temperaturas

---

## **Conclusiones**

1. **Calidad de datos:** El dataset tiene estructura consistente y cobertura temporal completa
2. **Resolución temporal:** La resolución horaria es adecuada para análisis energéticos
3. **Validez del modelo:** Los resultados del sistema fotovoltaico son físicamente coherentes
4. **Implementación exitosa:** Todas las funciones trabajan correctamente con manejo de errores robusto